## Principal Component Analysis Notebook
F. Burkholder

### Set-up

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# use ggplot style
plt.style.use('ggplot')

In [ ]:
# respect for the audience
import matplotlib
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 16}

# example of keyword argument unpacking
matplotlib.rc('font', **font)

### Show an example of an orthogonal transformation (a rotation)
Conceptually how PCA takes potentially correlated features and tranforms them into uncorrelated features.

In [ ]:
# create a simple linear dataset
m = 2 # slope
b = 0 # intercept
x1 = np.random.uniform(-5, 5, size = 20)
x2 = m * x1 + b

# reshape to column vectors or arbitrary length
x1 = x1.reshape((-1,1)) 
x2 = x2.reshape((-1,1))

# make it into a matrix
X = np.hstack((x1, x2))

print("x1\tx2")
for x1, x2 in X:
    print("{0:0.1f}\t{1:0.1f}".format(x1, x2))

In [ ]:
# plot it
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(X[:,0], X[:,1], marker='o', markersize=10, color='k', linewidth=0.5, linestyle='-')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
low = X.min() - 1
high = X.max() + 1
ax.set_xlim([low, high])
ax.set_ylim([low, high])
ax.set_aspect('equal');

In [ ]:
# determine its angle from the x-axis
theta = np.arctan(m)
print("The line is {0:0.2f} radians from the x1-axis, or {1:0.1f} degrees.".format(theta, np.degrees(theta)))

### Perform "orthogonal transformation" of the data, namely a [rotation.](https://en.wikipedia.org/wiki/Rotation_matrix)

In [ ]:
# define the rotation matrix, R
# use -theta to rotate it back to the X1 axis
R = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta), np.cos(theta)]])
print(R)

In [ ]:
# rotate the data
XR = np.dot(X, R)

In [ ]:
# plot XR and X
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(X[:,0], X[:,1], marker='o', markersize=10, color='k', linewidth=0.5, linestyle='-', label='X')
ax.plot(XR[:,0], XR[:,1], marker='o', markersize=10, color='b', linewidth=0.5, linestyle='-', label='XR')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
low = min(X.min(), XR.min()) - 1
high = max(X.max(), XR.max()) + 1
ax.set_xlim([low, high])
ax.set_ylim([low, high])
ax.legend()
ax.set_aspect('equal');

Note that with this transformation, data that once required feature x1 and x2 to describe its variance now has all of its variance described by the new x1R axis.  So dimensionality has been reduced, but all the variance in the data has been maintained.  We could build a model using only the one feature and it should perform just as well (and perhaps better), than the two feature model.

## How-to-PCA
1) Standardize columns  
2) Create covariance (correlation if standardized) matrix  
3) Find the eigenvectors and eigenvalues of the covariance/correlation matrix  
4) The eigenvectors are the principal components  

Will demonstrate using Numpy and Sklearn

## Numpy

In [ ]:
print(X)

### Standardize

In [ ]:
print("Column means: ", X.mean(axis=0)) # average of the columns
print("Column stddevs: ", X.std(axis=0, ddof=1)) # std dev. of the columns

In [ ]:
# standardize using numpy
X_std = (X - X.mean(axis=0))/X.std(axis=0, ddof=1)
print(X_std) #ugh, so many digits
print()
print(np.around(X_std, 2)) # better

In [ ]:
# check that column means are 0, standard deviation of 1
print(X_std.mean(axis=0))
print(X_std.std(axis=0, ddof=1))

In [ ]:
# could also use sklearn's standard scalar
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(X)
print("The means of the columns in X are: {0}".format(scaler.mean_.round(2)))
print("The standard deviations of the columns in X are: {0}".format(np.sqrt(scaler.var_).round(2)))

In [ ]:
# make sure data is standardized
X_std_ss = scaler.transform(X)
print(X_std_ss.mean(axis=0))
print(X_std_ss.std(axis=0, ddof=1)) #wat

In [ ]:
print(X_std_ss.std(axis=0, ddof=0)) #oh

Apparently standard scalar calculates variance only for a population, not a sample.

In [ ]:
# Ok back from standardizing
# plot it
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.scatter(X_std[:,0], X_std[:,1], marker='o', color='k')
ax.set_xlabel('x1_std')
ax.set_ylabel('x2_std')
low = X_std.min() - 1
high = X_std.max() + 1
ax.set_xlim([low, high])
ax.set_ylim([low, high])
ax.set_aspect('equal');

### Create covariance/correlation matrix
Covariance if just de-meaned, correlation if standardized too (more typical).

In [ ]:
N = X.shape[0]
A = 1/(N-1)*np.dot(X_std.T, X_std)

In [ ]:
print(A)

### Find the eigenvectors and eigenvalues (with numpy)
See [documentation](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.eig.html).  Structure of the returned eigenvectors is not intuitive.

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(A)

In [ ]:
eig_vals

In [ ]:
eig_vecs

In [ ]:
# will keep only the first eigenvector, associated with eigenvalue 2
pc1 = np.array([[eig_vecs[0][0], eig_vecs[1][0]]]).T
print(pc1)

See [here](https://stats.stackexchange.com/questions/229092/how-to-reverse-pca-and-reconstruct-original-variables-from-several-principal-com) for reconstructing your data matrix from principal components manually (in numpy).

## Use sklearn - example on the Iris dataset
Iris dataset described [here](https://en.wikipedia.org/wiki/Iris_flower_data_set)

In [ ]:
# Adapted from code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
from sklearn import datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features (there are a total of 4)
y = iris.target

print("X\t\ty")
for rowX, rowy in zip(X,y):
    print("{0}\t{1}".format(rowX, rowy))

In [ ]:
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

fig, ax = plt.subplots(figsize=(8,6))

# Plot the points
ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')
ax.set_xlabel('Sepal length')
ax.set_ylabel('Sepal width')
ax.set_title('Plotting first two columns of data')
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xticks(())
ax.set_yticks(());

Above, when we just used the first two columns of data, we didn't see much separation of the classes in our 2d plot.  If we had done PCA first, where we used PCA to reduce it down to two dimensions, maybe we'd see more separation...

In [ ]:
# see a 2d representation of the data using PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)


print("First 3 rows of the iris data:")
print("X1\tX2\tX3\tX4")
for i in range(3):
    print("{0}\t{1}\t{2}\t{3}".format(iris.data[i,0], iris.data[i,1], iris.data[i,2], iris.data[i,3]))

X_scaled = scaler.fit_transform(iris.data) # standardize data

pca = PCA(n_components=2) #pca object
X_pca = pca.fit_transform(X_scaled) # from 4 features to 2 PCA features

print("\nData after PCA into 2 components")
print("PC1\tPC2")
for i in range(3):
    print("{0:0.1f}\t{1:0.1f}".format(X_pca[i,0], X_pca[i,1]))

In [ ]:
# How are each of the 2 principal components defined?
# Here are the loadings (how the original features load on to the principal components)
pca.components_.round(2)

The 1st principal component (PC1) is 0.52 $\cdot$ X1 + -0.26 $\cdot$ X2 + 0.58 $\cdot$ X3 + 0.57 $\cdot$ X4  
The 2nd principal component (PC2) is 0.37 $\cdot$ X1 + 0.93 $\cdot$ X2 + 0.02 $\cdot$ X3 + 0.07 $\cdot$ X4

In [ ]:
# Are these components orthogonal?  (They should be!)
np.dot(pca.components_[0], pca.components_[1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y,
           cmap=plt.cm.Set1, edgecolor='k', s=40)
ax.set_title("First two PCA directions")
ax.set_xlabel("1st eigenvector (PC1)")
ax.set_ylabel("2nd eigenvector (PC2)");

By using features derived from linear combinations of all the features, this PCA 2d representation does a better job showing the structure in the data (the separations of the classes) than just using any two features.  

But did we make a sacrifice by going down to two dimensions (from four) using PCA?

In [ ]:
evr = pca.explained_variance_ratio_
print(evr)
print("The 2 principal components explain {0:0.1f}%"
      " of the variance in the original data.".format(evr.sum()*100))

^ That's really good!  A rule of thumb is to pick your number of components to explain at least 90% of the variance in the data.

### Scree plots (how to pick the number of principal components)

In [ ]:
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn import decomposition, datasets

pca = decomposition.PCA() # not setting number of components, which means 
                          # we keep them all!

digits = datasets.load_digits() # using MNIST
X_digits = digits.data
y_digits = digits.target

print("There are {0} rows of data.".format(X_digits.shape[0]))

In [ ]:
# look at a digit and target
img = 11
print(X_digits[img])
print("\nThe images are {0} in shape.".format(X_digits[img].shape))
print("\nEach value in the image is of type {0}".format(type(X_digits[img][0])))
print("(Though they look a lot like 4 bit numbers.)")
plt.imshow(X_digits[img].reshape((8,8)), cmap='gray')
print("\nNumber in image: ", y_digits[img])

In [ ]:
# not going to scale the images, because all pixel intensities
# are already on the same scale

pca.fit(X_digits)

# plot explained variance ratio in a scree plot
plt.figure(1, figsize=(8, 6))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2, color='red')
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_');

In [ ]:
total_variance = np.sum(pca.explained_variance_)
cum_variance = np.cumsum(pca.explained_variance_)
prop_var_expl = cum_variance/total_variance

fig, ax = plt.subplots(figsize=(8,6))
ax.plot(prop_var_expl, color='red', linewidth=2, label='Explained variance')
ax.axhline(0.9, label='90% goal', linestyle='--', color="black", linewidth=1)
ax.set_ylabel('cumulative prop. of explained variance')
ax.set_xlabel('number of principal components')
ax.legend();

### Around 20 principal components explains about 90% of the variance in the handwritten digits.
So, we could train a model on just those 20 features, instead of the original 64, and do as well or maybe better on unseen data!